In [32]:
from common_functions import get_secret, ret_metabase, google_sheets, snowflake_query, upload_dataframe_to_snowflake, dwh_query
from datetime import datetime, timedelta
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO

In [29]:

import jwt
fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
fintech_service_account_emailname= fintech_service_account["email_name"]
fintech_service_account_emailpass= fintech_service_account["email_password"]
payload = {
    "client_id":"admin-portal",
    "grant_type":"password",
    "username":fintech_service_account_emailname,
    "password":fintech_service_account_emailpass 
}


r = requests.post("https://sso.maxabsa.com/auth/realms/MaxABKenya/protocol/openid-connect/token",
    headers={"Content-Type":"application/x-www-form-urlencoded"},
    data=payload
)
fullJwtResponse = r.json()
# jwt_access_token = fullJwtResponse['access_token']

r.text




'{"error":"invalid_grant","error_description":"Invalid user credentials"}'

In [30]:
fintech_service_account

{'email_name': 'fintech-lambda@maxab.io',
 'email_password': 'q0Rkh%ONKf9@bznHU#',
 'personal_email': 'mahmoud.alheeny@maxab.io',
 'personal_password': 'MMGAFintech123!',
 'mp_password': '31Tgt11lSj%RqE4#CX'}

In [6]:
df = google_sheets("Telesales_daily_plan", "Source_sheet", "get")

/home/ec2-user/service_account_key.json


In [39]:
df["inserted_at"] = pd.to_datetime(df["inserted_at"], errors="coerce")

print("👉 Before filling (null rows):")
print(df[df["inserted_at"].isna()])

# Fill nulls with the fixed datetime
df["inserted_at"].fillna(pd.to_datetime("2025-09-01 11:11:11"), inplace=True)

print("\n👉 After filling (null rows should be gone):")
print(df[df["inserted_at"].isna()])

👉 Before filling (null rows):
Empty DataFrame
Columns: [Retailer_id, Call_status, Comment, inserted_at]
Index: []

👉 After filling (null rows should be gone):
Empty DataFrame
Columns: [Retailer_id, Call_status, Comment, inserted_at]
Index: []


/tmp/ipykernel_8499/3394429320.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["inserted_at"].fillna(pd.to_datetime("2025-09-01 11:11:11"), inplace=True)


In [40]:
df

,Retailer_id,Call_status,Comment,inserted_at
0,198793.0,Not answered,NaN,2025-09-01 08:58:55
1,201793.0,Not answered,NaN,2025-09-01 08:59:44
2,198577.0,Answered,NaN,2025-09-01 09:00:39
3,29197.0,Not answered,NaN,2025-09-01 09:01:43
4,47826.0,Answered,NaN,2025-09-01 09:02:26
...,...,...,...,...
1439,55671.0,Answered,NaN,2025-09-01 21:00:34
1440,25456.0,Answered,NaN,2025-09-01 21:02:07
1441,216217.0,Answered,NaN,2025-09-01 21:02:50
1442,200052.0,Not answered,NaN,2025-09-01 21:03:46


In [45]:
data = snowflake_query("Kenya", "select * from MATERIALIZED_VIEWS.Telesales_Source_Sheet ", warehouse=None, columns=[], conn=None)
data

/home/ec2-user/service_account_key.json


,retailer_id,call_status,comment,inserted_at
0,187509,Not answered,None,2025-07-31 11:30:56
1,213303,Answered,None,2025-07-31 11:31:27
2,116957,Not answered,None,2025-07-31 11:32:23
3,144177,Answered,None,2025-07-31 11:32:53
4,201890,Answered,None,2025-07-31 11:34:41
...,...,...,...,...
53263,55671,Answered,nan,2025-09-01 21:00:34
53264,25456,Answered,nan,2025-09-01 21:02:07
53265,216217,Answered,nan,2025-09-01 21:02:50
53266,200052,Not answered,nan,2025-09-01 21:03:46


In [35]:
test = dwh_query("Egypt", "select * from MATERIALIZED_VIEWS.ecom_account_management", "read", columns=[], conn=None)
test

/home/ec2-user/service_account_key.json


,primary_key,main_system_id,agent_assigned,call_status,comment,invalid_retailer,inserted_at,call_later
0,1756404171973260873,260873,3220,Reachable,,false,2025-08-28 21:02:51,
1,1756404048344429523,429523,3220,Unreachable,,false,2025-08-28 21:00:48,
2,1756404006454590489,590489,3220,Unreachable,,false,2025-08-28 21:00:06,
3,175640411536477038,77038,3220,Reachable,,false,2025-08-28 21:01:55,
4,778400,778400,2643,Reachable,الو بس,false,2025-08-28 18:36:46,
...,...,...,...,...,...,...,...,...
9126,1756807209466411286,411286,2643,Reachable,معندوش اب,true,2025-09-02 13:00:09,false
9127,1756807221653195923,195923,3221,Reachable,,false,2025-09-02 13:00:21,false
9128,1756807236825862140,862140,639,Unreachable,,false,2025-09-02 13:00:36,false
9129,1756807274453289995,289995,639,Reachable,,false,2025-09-02 13:01:14,false


In [36]:
sql = """
ALTER TABLE MATERIALIZED_VIEWS.ecom_account_management 
ADD CONSTRAINT ecom_account_management_pk PRIMARY KEY (primary_key);
"""

dwh_query(
    "Egypt",
    sql,
    "write",
    columns=[],
    conn=None
)


/home/ec2-user/service_account_key.json


In [26]:
data = data.sort_values(by="inserted_at", ascending=False)


In [27]:
data.head(40)

,primary_key,main_system_id,agent_assigned,call_status,comment,invalid_retailer,inserted_at,call_later
0,None,64526,2643,None,None,None,2025-09-01 18:47:59,None
1,None,12176,3221,None,None,None,2025-09-01 17:54:00,None
2,None,227356,3221,None,None,None,2025-09-01 16:53:14,None
3,None,384489,2407,None,None,None,2025-09-01 15:53:18,None
4,None,708667,2407,None,None,None,2025-09-01 14:53:38,None
5,None,608313,2840,None,None,None,2025-09-01 13:53:10,None
6,None,572158,639,None,None,None,2025-09-01 12:52:28,None
7,None,571898,639,None,None,None,2025-09-01 11:53:32,None
9,1756404171973260873,260873,3220,Reachable,,false,2025-08-28 21:02:51,
12,175640411536477038,77038,3220,Reachable,,false,2025-08-28 21:01:55,


In [41]:
print(df.columns.tolist())

['Retailer_id', 'Call_status', 'Comment', 'inserted_at']


In [42]:
# Ensure Comment column is all strings
df["Comment"] = df["Comment"].astype(str)

# If you prefer keeping NaNs instead of 'nan' strings:
df["Comment"] = df["Comment"].astype("string")

# Also reset index to avoid the UserWarning
df = df.reset_index(drop=True)


In [43]:
upload_dataframe_to_snowflake("Kenya", df, "MATERIALIZED_VIEWS", "Telesales_Source_Sheet", "append", auto_create_table=True, conn=None)

/home/ec2-user/service_account_key.json


True

In [44]:
eg_data = ret_metabase("Egypt", 61992)
eg_data

/home/ec2-user/service_account_key.json


,تاريخ اليوم,كود التاجر,رقم التليفون,سحب,ايداع
0,2025-09-01,882460,1270063666,208530.00,0.0
1,2025-09-01,882998,1067047198,173275.50,0.0
2,2025-09-01,492307,1015387110,1305993.60,0.0
3,2025-09-01,893336,1094531447,12789.84,0.0
4,2025-09-01,885904,1207504206,199602.93,0.0
5,2025-09-01,576120,1104801394,8735461.71,0.0
6,2025-09-01,880933,1097306909,1191600.00,0.0
7,2025-09-01,880920,1552088812,779866.43,0.0
8,2025-09-01,492358,1010639902,823792.80,0.0
9,2025-09-01,880884,1029272337,476640.00,0.0
